In [9]:
from helpers.idf_helpers import *
from helpers.plot_helpers import *
from studies import run_study
from helpers.idf_helpers import *
import pandas as pd
import matplotlib.pyplot as plt
import ipywidgets as widgets
from ipywidgets import interact, interact_manual, SelectMultiple
from eppy.modeleditor import IDF
from IPython.display import display

import os

In [10]:
outputs = pd.read_excel('Output Variables.xlsx')

In [11]:
temp = outputs[outputs['Units']=='C']

In [12]:
@interact
def get_output(output = temp['Output Name']):
    return temp[temp['Output Name']==output]

interactive(children=(Dropdown(description='output', options=('Site Outdoor Air Drybulb Temperature', 'Site Ou…

In [13]:
var_list = ['Zone Mean Radiant Temperature',
            'Zone Mean Air Temperature',
            'Zone Operative Temperature',
            'Zone Mean Air Dewpoint Temperature',
            'Zone Outdoor Air Wetbulb Temperature', 
            'Zone Air Temperature',
            'Zone Thermostat Air Temperature',
            'Zone Thermostat Heating Setpoint Temperature',
            'Zone Thermostat Cooling Setpoint Temperature',
      '     Zone Adaptive Comfort Operative Temperature Set Point',
           'Zone Heat Index']

In [14]:
idffile = '/home/paula/Documentos/Doctorado/Desarrollo/eppyProject/input/simple_room_1_window_1_door.idf'
iddfile = '/usr/local/EnergyPlus-9-2-0/Energy+.idd'
epwfile = '/home/paula/Documentos/Doctorado/Desarrollo/eppyProject/input/Wheather file/FRA_Paris.Orly.071490_IWEC.epw'
idf2 = initialization(idffile, iddfile, epwfile)

In [15]:
@interact
def show_not_null_classes(classes = notnullobj(idf2)):
    return epobject_to_df(idf2.idfobjects[classes])

interactive(children=(Dropdown(description='classes', options=('VERSION', 'SIMULATIONCONTROL', 'BUILDING', 'TI…

https://www.ashrae.org/technical-resources/ashrae-handbook/ashrae-handbook-online

#### Simulationontrol:
Do_Zone_Sizing_Calculation Necesita del objejo Sizing:Zone para funcionar

Do_System_Sizing_Calculation Necesita del objejo Sizing:Sytem para funcionar

****En este caso estas dos no tienen sentido xq no voy a hacer un autosize de ningún sistema..m  así que no apolican****


#### Building:

...No tengo nada específicado. Quizás debería poner algo más..o no?? xq contra menos variables tenga quizás mejor?

#### SizingPerdiod:Designday

The design day input describes the parameters to effect a “design day” simulation, often used for
load calculations or sizing equipment. Using the values in these fields, EnergyPlus “creates” a
complete days’ worth of weather data (air temperatures, solar radiation, etc.)

 Wetbulb bulbo húmedo (temperatura de bulbo húmedo)
 
 DewPoint: Punto de rocío
 
 #### Runperiod
 
 daylight saving period may be entered to be applied to weather
file run periods.Se refiere al periodo de cambio horario

#### Materials

Especifica las características físicas de los materiales presentes en la construcción
- grosor.
- Densidad
- conductividad
- calor específico

#### Materia No mass

Se usa para describir materiales opacos (supongo materiales usados para la  isolación)
 
 #### Material:AirGap
 
This material is used to describe the air gap in an opaque construction element.

#### Window material

...no hay nada en este objeto.... eso es raro





In [16]:
idf = idf2
objs_to_delete=[]
scriptname='variable_study'
vars_name=var_list
objs_to_delete = ['PEOPLE', 'LIGHTS', 'ELECTRICEQUIPMENT']

## tengo que hacer cambios para que esta función también funcione desde jupyter notebook

for obj in objs_to_delete:
    for o in idf.idfobjects[obj]:
        idf.removeidfobject(o)
# TODO hacer esto general
# hacer un remove de las variables que ya estaban

for n in range(len(vars_name)):
    idf.newidfobject('OUTPUT:VARIABLE')
    idf.idfobjects['OUTPUT:VARIABLE'][n].Variable_Name = vars_name[n]
# if idf.idfobjects['HVACTemplate:Thermostat']:
#     schedule_name = idf.idfobjects['HVACTemplate:Thermostat'][0].Heating_Setpoint_Schedule_Name
#     idf.newidfobject('OUTPUT:VARIABLE')
#     idf.idfobjects['OUTPUT:VARIABLE'][-1].Key_Value = schedule_name
#     idf.idfobjects['OUTPUT:VARIABLE'][-1].Variable_Name = 'Schedule Value'

# newdir = os.path.basename(__file__).replace('.py','')


foldername = scriptname.replace('.py', '')

try:
    os.mkdir(foldername)
except FileExistsError:
    print("Directory already exists")
abs_newdir = '/home/paula/Documentos/Doctorado/Desarrollo/eppyProject/variable_study'
#idf.saveas(abs_newdir + '/idffile.idf')
idf.run(output_directory=abs_newdir)

Directory already exists

/usr/local/EnergyPlus-9-2-0/energyplus --idd /usr/local/EnergyPlus-9-2-0/Energy+.idd --output-directory /home/paula/Documentos/Doctorado/Desarrollo/eppyProject/variable_study --weather /home/paula/Documentos/Doctorado/Desarrollo/eppyProject/input/Wheather file/FRA_Paris.Orly.071490_IWEC.epw /home/paula/Documentos/Doctorado/Desarrollo/eppyProject/in.idf



In [17]:
%%bash
cp studies/ReadVarsESO variable_study/


In [20]:
%%bash
ls variable_study/
cd variable_study/
./ReadVarsESO

eplusout.audit
eplusout.bnd
eplusout.csv
eplusout.eio
eplusout.end
eplusout.err
eplusout.eso
eplusout.expidf
eplusout.mdd
eplusout.mtd
eplusout.rdd
eplusout.shd
eplusout.sql
eplusssz.csv
eplustbl.csv
eplustbl.htm
epluszsz.csv
idffile.idf
readvars.audit
ReadVarsESO
sqlite.err
 ReadVarsESO program starting.
 ReadVars Run Time=00hr 00min  0.90sec
 ReadVarsESO program completed successfully.


In [27]:
data = pd.read_csv('variable_study/eplusout.csv')

In [28]:
data

,Date/Time,ATTIC_UNIT1:Zone Outdoor Air Wetbulb Temperature [C](Hourly),CRAWLSPACE_UNIT1:Zone Outdoor Air Wetbulb Temperature [C](Hourly),LIVING_UNIT1:Zone Mean Radiant Temperature [C](Hourly),ATTIC_UNIT1:Zone Mean Radiant Temperature [C](Hourly),CRAWLSPACE_UNIT1:Zone Mean Radiant Temperature [C](Hourly),LIVING_UNIT1:Zone Mean Air Temperature [C](Hourly),LIVING_UNIT1:Zone Operative Temperature [C](Hourly),LIVING_UNIT1:Zone Mean Air Dewpoint Temperature [C](Hourly),ATTIC_UNIT1:Zone Mean Air Temperature [C](Hourly),...,LIVING_UNIT1:Zone Thermostat Heating Setpoint Temperature [C](Hourly),LIVING_UNIT1:Zone Thermostat Cooling Setpoint Temperature [C](Hourly),ATTIC_UNIT1:Zone Air Temperature [C](Hourly),ATTIC_UNIT1:Zone Thermostat Air Temperature [C](Hourly),ATTIC_UNIT1:Zone Thermostat Heating Setpoint Temperature [C](Hourly),ATTIC_UNIT1:Zone Thermostat Cooling Setpoint Temperature [C](Hourly),CRAWLSPACE_UNIT1:Zone Air Temperature [C](Hourly),CRAWLSPACE_UNIT1:Zone Thermostat Air Temperature [C](Hourly),CRAWLSPACE_UNIT1:Zone Thermostat Heating Setpoint Temperature [C](Hourly),CRAWLSPACE_UNIT1:Zone Thermostat Cooling Setpoint Temperature [C](Hourly)
0,01/21 01:00:00,-20.027231,-19.990250,17.604582,-21.084830,10.757133,22.220000,19.912291,-12.325947,-20.994430,...,22.22,23.88,-20.994430,23.0,0.0,0.0,7.236737,23.0,0.0,0.0
1,01/21 02:00:00,-20.027231,-19.990250,17.557140,-21.085198,10.756226,22.220000,19.888570,-12.538038,-20.994726,...,22.22,23.88,-20.994726,23.0,0.0,0.0,7.236106,23.0,0.0,0.0
2,01/21 03:00:00,-20.027231,-19.990250,17.531847,-21.086256,10.754273,22.220000,19.875924,-12.716005,-20.995672,...,22.22,23.88,-20.995672,23.0,0.0,0.0,7.234612,23.0,0.0,0.0
3,01/21 04:00:00,-20.027231,-19.990250,17.516595,-21.087674,10.751860,22.220000,19.868298,-12.854737,-20.996960,...,22.22,23.88,-20.996960,23.0,0.0,0.0,7.232725,23.0,0.0,0.0
4,01/21 05:00:00,-20.027231,-19.990250,17.517782,-21.089038,10.749654,22.220000,19.868891,-12.948231,-20.998230,...,22.22,23.88,-20.998230,23.0,0.0,0.0,7.230974,23.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
8803,12/31 20:00:00,-2.192288,-2.155308,19.976866,-3.158890,15.433850,22.220032,21.098449,-0.188404,-3.084478,...,22.22,23.88,-3.084478,23.0,0.0,0.0,13.855374,23.0,0.0,0.0
8804,12/31 21:00:00,-2.077899,-2.040919,19.955639,-3.081608,15.392908,22.220032,21.087836,-0.096861,-2.998932,...,22.22,23.88,-2.998932,23.0,0.0,0.0,13.828965,23.0,0.0,0.0
8805,12/31 22:00:00,-1.739443,-1.702463,19.906214,-3.101045,15.398554,22.220032,21.063123,0.011862,-3.004268,...,22.22,23.88,-3.004268,23.0,0.0,0.0,13.907231,23.0,0.0,0.0
8806,12/31 23:00:00,-0.979139,-0.942158,19.809661,-3.079296,15.461850,22.220032,21.014846,0.250830,-2.977634,...,22.22,23.88,-2.977634,23.0,0.0,0.0,14.011870,23.0,0.0,0.0


In [1]:
data = eso_to_ts('variable_study/eplusout.csv')

NameError: name 'eso_to_ts' is not defined

In [22]:
winterperiod = ['2000-01-08','2000-01-10']
summerperiod= ['2000-08-08','2000-08-10']

In [23]:
datawinter = data[winterperiod[0]:winterperiod[1]]

NameError: name 'data' is not defined

In [ ]:
datawinter.plot(title = 'invierno',figsize = (12,5),grid =True).legend(loc = 'upper left',bbox_to_anchor=(1.05, 1))